In [4]:
import torch
from torch import nn

def greedy_sample(logits):
    batch_size, seq_len, _ = logits.size()
    max_values, max_indices = logits.max(dim=2)
    return max_indices

def ppl(sequences, log_probs):
    batch_size, seq_len = sequences.size()
    seq_log_probs = torch.zeros_like(sequences).float()
    for b in range(batch_size):
        for t in range(seq_len):
            idx = sequences[b, t].item()
            seq_log_probs[b, t] = log_probs[b, t, idx].item()
    return seq_log_probs.sum()

def perplexity(truths, sampled, log_probs):
    batch_size, seq_len, vocab_size = log_probs.size()
    _ppl = {
        'ground-truth': ppl(truths, log_probs).mean(),
        'sampled': ppl(sampled, log_probs).mean(),
    }
    return _ppl

In [5]:
#clipped_value --- paramete rfor ClippedAdfam 

class RL(nn.Module):
    def __init__(self, gamma, clip_value):
        super().__init__()
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.log_sigmoid = torch.nn.LogSigmoid()

    def forward(self, log_probs, logits, weight, baseline=None):
        batch_size, seqlen, _ = logits.size()
        rewards = self.log_sigmoid(logits).squeeze(2)

        cumulative_rewards = []
        for t in range(seqlen):
            cum_value = rewards.new_zeros(batch_size)
            for s in range(t, seqlen):
                exp = float(s-t)
                k = (self.gamma ** exp)
                cum_value +=  k * weight[:, s]  * rewards[:, s]
            cumulative_rewards.append(cum_value)

        cumulative_rewards = torch.stack(cumulative_rewards, dim=1)

        if baseline is not None:
            baseline = baseline.squeeze(2)
            advantages = cumulative_rewards - baseline
        else:
            advantages = cumulative_rewards

        advantages = advantages - advantages.mean(dim=0)
        advantages = advantages.clamp(-1*self.clip_value, self.clip_value)
        
        generator_objective = (advantages * log_probs).sum(dim=0)
        return (generator_objective, cumulative_rewards.clone())